In [33]:
import numpy as np
import pickle
import cv2
import time

In [139]:
def ReLU(data):
	if(data >= 0):
		return data
	else: 
		return 0
def ReLU_2d(data):
	row, col = data.shape
	data_ret = np.zeros((row, col))

	for i in range(row):
		for j in range(col):
			data_ret[i, j] = ReLU(data[i, j])
	
	return data_ret

def ReLU_3d(data):
	row, col, num_filter = data.shape
	data_ret = np.zeros((row, col, num_filter))

	for i in range(num_filter):
		data_ret[:, :, i] = ReLU_2d(data[:, :, i])

	return data_ret

def Sigmoid(data):
	return 1 / (1 + math.exp(-data))

def Sigmoid_2d(data):
	row, col = data.shape
	data_ret = np.zeros((row, col))

	for i in range(row):
		for j in range(col):
			data_ret[i, j] = Sigmoid(data[i, j])
	
	return data_ret

def max_pooling(matrix):
	row, col = matrix.shape

	max_value = 0

	for i in range(row):
		for j in range(col):
			if(matrix[i][j] > max_value):
				max_value = matrix[i][j]
	return max_value

def avg_pooling(matrix):
	row, col = matrix.shape

	sum_value = 0

	for i in range(row):
		for j in range(col):
				sum_value = sum_value + matrix[i][j]

	avg_value = sum_value/(col*row)
	return avg_value

def median_pooling(matrix):
	row, col = matrix.shape

	matrix = np.reshape(matrix, (1, row*col))

	matrix = np.sort(matrix)

	num_element = row*col

	median_index = int(num_element/2)

	return matrix[0, median_index]

def pooling_2d(x_matrix, dim_filter, stride, padding, pooling_option):
	x_row, x_col = x_matrix.shape
	filter_row, filter_col = dim_filter

	y_row = int(((x_row - filter_row + 2*padding)/stride) + 1)
	y_col = int(((x_col - filter_col + 2*padding)/stride) + 1)

	if(padding >= 1):
		temp_matrix = x_matrix
		x_row = x_row + 2*padding
		x_col = x_col + 2*padding
		x_matrix = np.zeros((x_row, x_col))
		x_matrix[0+padding:x_row-padding, 0+padding:x_col-padding] = temp_matrix

	y = np.zeros((y_row, y_col))

	index_row = np.arange(start=0, stop=x_row-filter_row+1, step=stride)
	index_col = np.arange(start=0, stop=x_col-filter_col+1, step=stride)
	for i in range(len(index_row)):
		for j in range(len(index_col)):
			sub_matrix = x_matrix[index_row[i]:index_row[i]+filter_row, index_col[j]:index_col[j]+filter_col]

			if(pooling_option == 0):
				y[i, j] = max_pooling(sub_matrix)
			elif(pooling_option == 1):
				y[i, j] = avg_pooling(sub_matrix)
			else:
				y[i, j] = median_pooling(sub_matrix)
			
	return y

def pooling_3d(x_matrix, dim_filter, stride, padding, pooling_option):
	x_row, x_col, num_filter = x_matrix.shape
	filter_row, filter_col = dim_filter

	y_row = int(((x_row - filter_row + 2*padding)/stride) + 1)
	y_col = int(((x_col - filter_col + 2*padding)/stride) + 1)

	y = np.zeros((y_row, y_col, num_filter))

	for i in range(num_filter):
		if(pooling_option == 0):
			y[:, :, i] = pooling_2d(x_matrix[:, :, i], dim_filter, stride, padding, pooling_option)
		elif(pooling_option == 1):
			y[:, :, i] = pooling_2d(x_matrix[:, :, i], dim_filter, stride, padding, pooling_option)
		else:
			y[:, :, i] = pooling_2d(x_matrix[:, :, i], dim_filter, stride, padding, pooling_option)

	return y

def conv(x_matrix, filter_matrix):
	row, col = x_matrix.shape
	y = np.zeros((row, col))

	for i in range(row):
		for j in range(col):
			y[i, j] = x_matrix[i, j]*filter_matrix[i, j]
	
	return y.sum()
	
def conv2(x_matrix, filter_matrix, stride, padding):
	x_row, x_col = x_matrix.shape
	filter_row, filter_col = filter_matrix.shape 

	y_row = int(((x_row - filter_row + 2*padding)/stride) + 1)
	y_col = int(((x_col - filter_col + 2*padding)/stride) + 1)

	if(padding >= 1):
		temp_matrix = x_matrix
		x_row = x_row + 2*padding
		x_col = x_col + 2*padding
		x_matrix = np.zeros((x_row, x_col))
		x_matrix[0+padding:x_row-padding, 0+padding:x_col-padding] = temp_matrix

	y = np.zeros((y_row, y_col))

	index_row = np.arange(start=0, stop=x_row-filter_row+1, step=stride)
	index_col = np.arange(start=0, stop=x_col-filter_col+1, step=stride)
	for i in range(len(index_row)):
		for j in range(len(index_col)):
			sub_matrix = x_matrix[index_row[i]:index_row[i]+filter_row, index_col[j]:index_col[j]+filter_col]
			y[i, j] = conv(sub_matrix, filter_matrix)
	return y

def conv3(x_matrix, filter_matrix, stride, padding):
	row_filter, col_filter, num_sub_filter, num_filter = filter_matrix.shape 				# row_filter_matrix = col_filter_matrix
	row_x, col_x, num_x = x_matrix.shape 													# row_x = col_x
																							# num_sub_filter = num_x
	y_row = int(((row_x - row_filter + 2*padding)/stride) + 1)
	y_col = int(((col_x - col_filter + 2*padding)/stride) + 1)
	
	data = np.zeros((y_row, y_col, num_filter))
	for i in range(num_filter):
		data_temp = np.zeros((y_row, y_col))

		for j in range(num_sub_filter):
			data_temp = data_temp + conv2(x_matrix[:, :, j], filter_matrix[:, :, j, i], stride, padding)
		
		data[:, :, i] = data_temp

	return data

def add_bias_after_conv(conv_result, bias):
	row, col, num_filter = conv_result.shape

	for i in range(num_filter):
		conv_result[:, :, i] = conv_result[:, :, i] + bias[i, :]

	return conv_result

def Feed_forward_Lenet_5(image, conv0_filter, bias_conv0, conv1_filter, bias_conv1, W0, W1, W2, B0, B1, B2, max_result_conv0, min_result_conv0, max_result_conv1, min_result_conv1, max_pooling0, min_pooling0, max_pooling1, min_pooling1, max_hd0, min_hd0, max_hd1, min_hd1, max_out_bf_act, min_out_gf_act):
	stride = 1
	padding = 0
	stride_pooling = 2
	padding_pooling = 0
	pooling_option = 1

	# Convolution layer 0
	conv0_result = conv3(image, conv0_filter, stride, padding)
	add_bias_after_conv0_bf_relu = add_bias_after_conv(conv0_result, bias_conv0)
	add_bias_after_conv0 = ReLU_3d(add_bias_after_conv0_bf_relu)
	pooling0 = pooling_3d(add_bias_after_conv0, (2, 2), stride_pooling, padding_pooling, pooling_option)
	

	# Convolution layer 1
	conv1_result = conv3(pooling0, conv1_filter, stride, padding)
	add_bias_after_conv1_bf_relu = add_bias_after_conv(conv1_result, bias_conv1)
	add_bias_after_conv1 = ReLU_3d(add_bias_after_conv1_bf_relu)
	pooling1 = pooling_3d(add_bias_after_conv1, (2, 2), stride_pooling, padding_pooling, pooling_option)
	
	# Fully connected
	flatten = pooling1.reshape(-1, 1)
	hd_layer0_bf_relu = np.dot(W0, flatten) + B0
	hd_layer0 = ReLU_2d(hd_layer0_bf_relu)
	hd_layer1_bf_relu = np.dot(W1, hd_layer0) + B1
	hd_layer1 = ReLU_2d(hd_layer1_bf_relu)
	output_bf_sigmoid = np.dot(W2, hd_layer1) + B2
	output = Sigmoid_2d(output_bf_sigmoid)
	
	max_data = 0
	result = 0
	for i in range(output.shape[0]):
		if(output[i, :] > max_data):
			max_data = output[i, :]
			result = i
	
	if(max_result_conv0 < np.amax(add_bias_after_conv0_bf_relu)):
		max_result_conv0 = np.amax(add_bias_after_conv0_bf_relu)
	if(min_result_conv0 > np.amin(add_bias_after_conv0_bf_relu)):
		min_result_conv0 = np.amin(add_bias_after_conv0_bf_relu)
	
	if(max_result_conv1 < np.amax(add_bias_after_conv1_bf_relu)):
		max_result_conv1 = np.amax(add_bias_after_conv1_bf_relu)
	if(min_result_conv1 > np.amin(add_bias_after_conv1_bf_relu)):
		min_result_conv1 = np.amin(add_bias_after_conv1_bf_relu)
	
	if(max_pooling0 < np.amax(pooling0)):
		max_pooling0 = np.amax(pooling0)
	if(min_pooling0 > np.amin(pooling0)):
		min_pooling0 = np.amin(pooling0)

	if(max_pooling1 < np.amax(pooling1)):
		max_pooling1 = np.amax(pooling1)
	if(max_pooling1 > np.amin(pooling1)):
		min_pooling1 = np.amin(pooling1)
	
	if(max_hd0 < np.amax(hd_layer0_bf_relu)):
		max_hd0 = np.amax(hd_layer0_bf_relu)
	if(min_hd0 > np.amin(hd_layer0_bf_relu)):
		min_hd0 = np.amin(hd_layer0_bf_relu)
	
	if(max_hd1 < np.amax(hd_layer1_bf_relu)):
		max_hd1 = np.amax(hd_layer1_bf_relu)
	if(min_hd1 > np.amin(hd_layer1_bf_relu)):
		min_hd1 = np.amin(hd_layer1_bf_relu)
	
	if(max_out_bf_act < np.amax(output_bf_sigmoid)):
		max_out_bf_act = np.amax(output_bf_sigmoid)
	if(min_out_gf_act > np.amin(output_bf_sigmoid)):
		min_out_gf_act = np.amin(output_bf_sigmoid)

	return result, max_result_conv0, min_result_conv0, max_result_conv1, min_result_conv1, max_pooling0, min_pooling0, max_pooling1, min_pooling1, max_hd0, min_hd0, max_hd1, min_hd1, max_out_bf_act, min_out_gf_act

def Load_Conv_and_W_and_Bias(data_path, load_option, conv_shape):
	temp_data = np.loadtxt(data_path)

	if(load_option == 0): # ================== Conv
		row, col, num_sub_filter, num_filter = conv_shape # row = col
		data = np.zeros((row, col, num_sub_filter, num_filter))

		index = 0 # maximum index = num_sub_filter * num_filter
		for m in range(num_filter):
			for k in range(num_sub_filter):
				data[:, :, k, m] = temp_data[index * row: index * row + 5, :]
				index = index + 1
		return data
	elif(load_option == 1): # ================ W
		return temp_data
	else: # ============================= Bias
		return temp_data.reshape(-1, 1)



In [37]:
Conv0_shape = (5, 5, 1, 6)
Conv1_shape = (5, 5, 6, 16)

Conv0_path = '../../Weight_and_bias_of_model_python/Conv0.txt'
Bias_conv0_path = '../../Weight_and_bias_of_model_python/Bias_conv0.txt'
Conv1_path = '../../Weight_and_bias_of_model_python/Conv1.txt'
Bias_conv1_path = '../../Weight_and_bias_of_model_python/Bias_conv1.txt'

W0_path = '../../Weight_and_bias_of_model_python/W0.txt'
W1_path = '../../Weight_and_bias_of_model_python/W1.txt'
W2_path = '../../Weight_and_bias_of_model_python/W2.txt'

B0_path = '../../Weight_and_bias_of_model_python/B0.txt'
B1_path = '../../Weight_and_bias_of_model_python/B1.txt'
B2_path = '../../Weight_and_bias_of_model_python/B2.txt'

In [38]:
pickle_in = open("../../Dataset/Data_test_10000.pickle","rb")
image_test = pickle.load(pickle_in)
pickle_in = open("../../Dataset/Label_test_10000.pickle","rb")
label_test = pickle.load(pickle_in)

Conv0 = Load_Conv_and_W_and_Bias(Conv0_path, 0, Conv0_shape)
Bias_conv0 = Load_Conv_and_W_and_Bias(Bias_conv0_path, 2, 0)
Conv1 = Load_Conv_and_W_and_Bias(Conv1_path, 0, Conv1_shape)
Bias_conv1 = Load_Conv_and_W_and_Bias(Bias_conv1_path, 2, 0)

W0 = Load_Conv_and_W_and_Bias(W0_path, 1, 0)
W1 = Load_Conv_and_W_and_Bias(W1_path, 1, 0)
W2 = Load_Conv_and_W_and_Bias(W2_path, 1, 0)

B0 = Load_Conv_and_W_and_Bias(B0_path, 2, 0)
B1 = Load_Conv_and_W_and_Bias(B1_path, 2, 0)
B2 = Load_Conv_and_W_and_Bias(B2_path, 2, 0)

In [39]:
print('\n============================== Weight in convolution layer ==============================\n')
print('Max weight in conv layer 0: ', np.amax(Conv0), '\nMin weight in conv layer 0: ', np.amin(Conv0))
print('Max weight in conv layer 1: ', np.amax(Conv1), '\nMin weight in conv layer 1: ', np.amin(Conv1))
print('\n============================== Bias in convolution layer ==============================\n')
print('Max bias in conv layer 0: ', np.amax(Bias_conv0), '\nMin bias in conv layer 0: ', np.amin(Bias_conv0))
print('Max bias in conv layer 1: ', np.amax(Bias_conv1), '\nMin bias in conv layer 1: ', np.amin(Bias_conv1))
print('\n============================== Weight in FC ==============================\n')
print('Max w in W0: ', np.amax(W0), '\nMin w in W0: ',  np.amin(W0))
print('Max w in W1: ', np.amax(W1), '\nMin w in W1: ',  np.amin(W1))
print('Max w in W2: ', np.amax(W2), '\nMin w in W2: ', np.amin(W2))
print('\n=================================== Bias in FC ===================================\n')
print('Max bias in B0: ', np.amax(B0), '\nMin bias in B0: ',  np.amin(B0))
print('Max bias in B1: ', np.amax(B1), '\nMin bias in B1: ',  np.amin(B1))
print('Max bias in B2: ', np.amax(B2), '\nMin bias in B2: ', np.amin(B2))


============================== Weight in convolution layer ==============================

Max weight in conv layer 0:  0.45626965 
Min weight in conv layer 0:  -0.6334114
Max weight in conv layer 1:  0.5986896 
Min weight in conv layer 1:  -0.9331202

============================== Bias in convolution layer ==============================

Max bias in conv layer 0:  0.03692436 
Min bias in conv layer 0:  -0.34630153
Max bias in conv layer 1:  0.020025745 
Min bias in conv layer 1:  -0.15468165

============================== Weight in FC ==============================

Max w in W0:  0.8224239 
Min w in W0:  -1.0181603
Max w in W1:  0.777616 
Min w in W1:  -0.7706447
Max w in W2:  0.46389437 
Min w in W2:  -0.84771866

=================================== Bias in FC ===================================

Max bias in B0:  0.1809927 
Min bias in B0:  -0.20980695
Max bias in B1:  0.18734296 
Min bias in B1:  -0.18926921
Max bias in B2:  0.23310512 
Min bias in B2:  -0.1263325


In [150]:
max_image = 1
min_image = -1

max_filter_conv0 = 0.64
min_filter_conv0 = -0.64
max_filter_conv1 = 0.94
min_filter_conv1 = -0.94

max_result_conv0 = 2.67
min_result_conv0 = -2.67
max_result_conv1 = 8.88
min_result_conv1 = -8.88

max_bias_conv0 = 0.35
min_bias_conv0 = -0.35
max_bias_conv1 = 0.16
min_bias_conv1 = -0.16

max_pooling0 = 1.75
min_pooling0 = -1.75
max_pooling1 = 2.84
min_pooling1 = -2.84

max_w0 = 1.02
min_w0 = -1.02
max_w1 = 0.78
min_w1 = -0.78
max_w2 = 0.85
min_w2 = -0.85

max_b0 = 0.21
min_b0 = -0.21
max_b1 = 0.19
min_b1 = -0.19
max_b2 = 0.24
min_b2 = -0.24

max_in = 2.84
min_in = -2.84
max_hd0 = 9.6
min_hd0 = -9.6
max_hd1 = 12.8
min_hd1 = -12.8
max_out_bf_act = 23.4
min_out_bf_act = 23.4
max_out = 1
min_out = -1

In [158]:
pre_time = time.time()
for i in range(10000):
    if(max_image < np.amax(image_test[i])):
        max_image = np.amax(image_test[i])
    if(min_image > np.amin(image_test[i])):
        min_image = np.amin(image_test[i])

delta_time = (time.time() - pre_time)
print('Time to handle: ', delta_time, 's')

print('\n============================== Image ==============================\n')
print('Max value: ', max_image, '\nMin value: ', min_image)

Time to handle:  0.18200039863586426 s

============================== Image ==============================

Max value:  1.0 
Min value:  0


In [151]:
num_test = 10000
num_right = 0

pre_time = time.time()

for i in range(num_test):
    if(i % 1000 == 0):
        print('Processing... ', i, 'th test')
    result, max_result_conv0, min_result_conv0, max_result_conv1, min_result_conv1, max_pooling0, min_pooling0, max_pooling1, min_pooling1, max_hd0, min_hd0, max_hd1, min_hd1, max_out_bf_act, min_out_gf_act = Feed_forward_Lenet_5(image_test[i], Conv0, Bias_conv0, Conv1, Bias_conv1, W0, W1, W2, B0, B1, B2, max_result_conv0, min_result_conv0, max_result_conv1, min_result_conv1, max_pooling0, min_pooling0, max_pooling1, min_pooling1, max_hd0, min_hd0, max_hd1, min_hd1, max_out_bf_act, min_out_gf_act)

    if(result == label_test[i]):
        num_right = num_right + 1

delta_time = (time.time() - pre_time)
print('Time to handle: ', delta_time, 's')

accuracy = num_right / num_test

print(accuracy)

max_in = max_pooling1
min_in = min_pooling1

print('\n============================== Result in convolution layer ==============================\n')
print('Max result in conv layer 0: ', max_result_conv0, '\nMin result in conv layer 0: ', min_result_conv0)
print('Max result in conv layer 1: ', max_result_conv1, '\nMin result in conv layer 1: ', min_result_conv1)
print('Max result pooling in layer 0: ', max_pooling0, '\nMin result pooling in layer 0: ', min_pooling0)
print('Max result pooling in layer 1: ', max_pooling1, '\nMin result pooling in layer 1: ', min_pooling1)
print('\n============================== Fully Connected ==============================\n')
print('Max value input: ', max_in, '\nMin value input: ', min_in)
print('Max value in hd 0: ', max_hd0, '\nMin value in hd 0: ', min_hd0)
print('Max value in hd 1: ', max_hd1, '\nMin value in hd 1: ', min_hd1)
print('Max value output before sigmoid: ', max_out_bf_act, '\nMin value output before sigmoid: ', min_out_gf_act)

Processing...  0 th test
Processing...  1000 th test
Processing...  2000 th test
Processing...  3000 th test
Processing...  4000 th test
Processing...  5000 th test
Processing...  6000 th test
Processing...  7000 th test
Processing...  8000 th test
Processing...  9000 th test
Time to handle:  7584.245477676392 s
0.9893

============================== Result in convolution layer ==============================

Max result in conv layer 0:  2.564670004597997 
Min result in conv layer 0:  -2.6674656270000003
Max result in conv layer 1:  4.5858582353704085 
Min result in conv layer 1:  -8.871344010236188
Max result pooling in layer 0:  1.7406618107390415 
Min result pooling in layer 0:  0
Max result pooling in layer 1:  2.838221029965695 
Min result pooling in layer 1:  0.0

============================== Fully Connected ==============================

Max value input:  2.838221029965695 
Min value input:  0.0
Max value in hd 0:  6.138875842161627 
Min value in hd 0:  -9.599711009649864
Max

In [ ]:
# ============================================ Bit-width ============================================
bit_width_image = 4
bit_width_filter_conv0 = 4
bit_width_filter_conv1 = 4
bit_width_conv0 = 4
bit_width_conv1 = 4
bit_width_bias_conv0 = 4
bit_width_bias_conv1 = 4
bit_width_pooling0 = 4
bit_width_pooling1 = 4
bit_width_input = bit_width_pooling1

bit_width_w0 = 4
bit_width_w1 = 4
bit_width_w2 = 4

bit_width_b0 = 4
bit_width_b1 = 4
bit_width_b2 = 4

bit_width_h0 = 4
bit_width_h1 = 4
bit_width_bf_sigmoid = 4
bit_width_output = 4

In [ ]:
# ============================================ Max-min quantization value ============================================
max_image_q = 2**(bit_width_image-1)-1
min_image_q = -2**(bit_width_image-1)
max_filter_conv0_q = 2**(bit_width_filter_conv0-1)-1
min_filter_conv0_q = -2**(bit_width_filter_conv0-1)
max_filter_conv1_q = 2**(bit_width_filter_conv1-1)-1
min_filter_conv1_q = -2**(bit_width_filter_conv1-1)
max_conv0_q = 2**(bit_width_conv0-1)-1
min_conv0_q = -2**(bit_width_conv0-1)
max_conv1_q = 2**(bit_width_conv1-1)-1
min_conv1_q = -2**(bit_width_conv1-1)
max_bias_conv0_q = 2**(bit_width_bias_conv0-1)-1
min_bias_conv0_q = -2**(bit_width_bias_conv0-1)
max_bias_conv1_q = 2**(bit_width_bias_conv1-1)-1
min_bias_conv1_q = -2**(bit_width_bias_conv1-1)
max_pooling0_q = 2**(bit_width_pooling0-1)-1
min_pooling0_q = -2**(bit_width_pooling0-1)
max_pooling1_q = 2**(bit_width_pooling1-1)-1
min_pooling1_q = -2**(bit_width_pooling1-1)

max_w0_q = 2**(bit_width_w0-1)-1
min_w0_q = -2**(bit_width_w0-1)
max_w1_q = 2**(bit_width_w1-1)-1
min_w1_q = -2**(bit_width_w1-1)
max_w2_q = 2**(bit_width_w2-1)-1
min_w2_q = -2**(bit_width_w2-1)

max_b0_q = 2**(bit_width_b0-1)-1
min_b0_q = -2**(bit_width_b0-1)
max_b1_q = 2**(bit_width_b1-1)-1
min_b1_q = -2**(bit_width_b1-1)
max_b2_q = 2**(bit_width_b2-1)-1
min_b2_q = -2**(bit_width_b2-1)

max_input_q = 2**(bit_width_input-1)-1
min_input_q = -2**(bit_width_input-1)
max_hd0_q = 2**(bit_width_h0-1)-1
min_hd0_q = -2**(bit_width_h0-1)
max_hd1_q = 2**(bit_width_h1-1)-1
min_hd1_q = -2**(bit_width_h1-1)
max_bf_sigmoid_q = 2**(bit_width_bf_sigmoid-1)-1
min_bf_sigmoid_q = -2**(bit_width_bf_sigmoid-1)
max_output_q = 2**(bit_width_output-1)-1
min_output_q = -2**(bit_width_output-1)

In [ ]:
# ============================================ Scale and offset for w1 & bias ============================================
# Quantied number = (X/Scale) + Offset
scale_image = (max_image - min_image)/(max_image_q - min_image_q)
scale_filter_conv0 = (max_filter_conv0 - min_filter_conv0)/(max_filter_conv0_q - min_filter_conv0_q)
scale_filter_conv1 = (max_filter_conv1 - min_filter_conv1)/(max_filter_conv1_q - min_filter_conv1_q)
scale_conv0 = (max_result_conv0 - min_result_conv0)/(max_conv0_q - min_conv0_q)
scale_conv1 = (max_result_conv1 - min_result_conv1)/(max_conv1_q - min_conv1_q)
scale_bias_conv0 = (max_bias_conv0 - min_bias_conv0)/(max_bias_conv0_q - min_bias_conv0_q)
scale_bias_conv1 = (max_bias_conv1 - min_bias_conv1)/(max_bias_conv1_q - min_bias_conv1_q)
scale_pooling0 = (max_pooling0 - min_pooling0)/(max_pooling0_q - min_pooling0_q)
scale_pooling1 = (max_pooling1 - min_pooling1)/(max_pooling1_q - min_pooling1_q)

scale_w0 = (max_w0 - min_w0)/(max_w0_q - min_w0_q)
scale_w1 = (max_w1 - min_w1)/(max_w1_q - min_w1_q)
scale_w2 = (max_w2 - min_w2)/(max_w2_q - min_w2_q)

scale_b0 = (max_b0 - min_b0)/(max_b0_q - min_b0_q)
scale_b1 = (max_b1 - min_b1)/(max_b1_q - min_b1_q)
scale_b2 = (max_b2 - min_b2)/(max_b2_q - min_b2_q)

scale_input = (max_in - min_in)/(max_input_q - min_input_q)
scale_hd0 = (max_hd0 - min_hd0)/(max_hd0_q - min_hd0_q)
scale_hd1 = (max_hd1 - min_hd1)/(max_hd1_q - min_hd1_q)
scale_bf_sigmoid = (max_out_bf_act - min_out_bf_act)/(max_bf_sigmoid_q - min_bf_sigmoid_q)
scale_output = (max_out - min_out)/(max_output_q - min_output_q)

offset_image = round((max_image*min_image_q - min_image*max_image_q)/(max_image - min_image))
offset_filter_conv0 = round((max_filter_conv0*min_filter_conv0_q - min_filter_conv0*max_filter_conv0_q)/(max_filter_conv0 - min_filter_conv0))
offset_filter_conv1 = round((max_filter_conv1*min_filter_conv1_q - min_filter_conv1*max_filter_conv1_q)/(max_filter_conv1 - min_filter_conv1))
offset_conv0 = round((max_result_conv0*min_conv0_q - min_result_conv0*max_conv0_q)/(max_result_conv0 - min_result_conv0))
offset_conv1 = round((max_result_conv1*min_conv1_q - min_result_conv1*max_conv1_q)/(max_result_conv1 - min_result_conv1))
offset_bias_conv0 = round((max_bias_conv0*min_bias_conv0_q - min_bias_conv0*max_bias_conv0_q)/(max_bias_conv0 - min_bias_conv0))
offset_bias_conv1 = round((max_bias_conv1*min_bias_conv1_q - min_bias_conv1*max_bias_conv1_q)/(max_bias_conv1 - min_bias_conv1))
offset_pooling0 = round((max_pooling0*min_pooling0_q - min_pooling0*max_pooling0_q)/(max_pooling0 - min_pooling0))
offset_pooling1 = round((max_pooling1*min_pooling1_q - min_pooling1*max_pooling1_q)/(max_pooling1 - min_pooling1))

offset_w0 = round((max_w0*min_w0_q - min_w0*max_w0_q)/(max_w0 - min_w0))
offset_w1 = round((max_w1*min_w1_q - min_w1*max_w1_q)/(max_w1 - min_w1))
offset_w2 = round((max_w2*min_w2_q - min_w2*max_w2_q)/(max_w2 - min_w2))

offset_b0 = round((max_b0*min_b0_q - min_b0*max_b0_q)/(max_b0 - min_b0))
offset_b1 = round((max_b1*min_b1_q - min_b1*max_b1_q)/(max_b1 - min_b1))
offset_b2 = round((max_b2*min_b2_q - min_b2*max_b2_q)/(max_b2 - min_b2))

offset_input = round((max_in*min_input_q - min_in*max_input_q)/(max_in - min_in))
offset_hd0 = round((max_hd0*min_hd0_q - min_hd0*max_hd0_q)/(max_hd0 - min_hd0))
offset_hd1 = round((max_hd1*min_hd1_q - min_hd1*max_hd1_q)/(max_hd1 - min_hd1))
offset_bf_sigmoid = round((max_out_bf_act*min_bf_sigmoid_q - min_out_bf_act*max_bf_sigmoid_q)/(max_out_bf_act - min_out_bf_act))
offset_output = round((max_out*min_output_q - min_out*max_output_q)/(max_out - min_out))

# ============================================ Scale for quantize ============================================
# X * Scale 
scale_calculate_convolution_0 = (scale_image*scale_filter_conv0)/scale_conv0
scale_for_pooling = scale_conv0/(scale_pooling0*4)
scale_calculate_convolution_1 = (scale_pooling0*scale_filter_conv1)/scale_conv1
scale_for_pooling = scale_conv1/(scale_pooling1*4)

scale_for_W0_x_Flatten = (scale_input*scale_w0)/scale_hd0
scale_for_W1_x_hd0 = (scale_hd0*scale_w1)/scale_hd1
scale_for_W2_x_hd1 = (scale_hd1*scale_w2)/scale_bf_sigmoid